In [41]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('../Data/data_handledMissingValues.csv')
df.drop('Unnamed: 0',inplace=True,axis=1)
df.head()

,State,NAICS,Term,NoEmp,NewExist,LowDoc,MIS_Status,SBA_GrPortion
0,IN,45.0,84,4,1.0,Y,P I F,0.80
1,IN,72.0,60,2,1.0,Y,P I F,0.80
2,IN,62.0,180,7,0.0,N,P I F,0.75
3,CT,33.0,120,19,0.0,N,P I F,0.75
4,FL,81.0,84,1,1.0,Y,P I F,0.80


### First I will transform the value in NAICS column from float to int type. And then I once again change the datatype to String. Eventually, I map each value of this column with an industry based on the given information about NAICS. 


In [42]:
df['NAICS']=df['NAICS'].astype(int)
df['NAICS']=df['NAICS'].astype(str)
df['NAICS'] = df['NAICS'].map({
    '11': 'Ag/For/Fish/Hunt',
    '21': 'Min/Quar/Oil_Gas_ext',
    '22': 'Utilities',
    '23': 'Construction',
    '31': 'Manufacturing',
    '32': 'Manufacturing',
    '33': 'Manufacturing',
    '42': 'Wholesale_trade',
    '44': 'Retail_trade',
    '45': 'Retail_trade',
    '48': 'Trans/Ware',
    '49': 'Trans/Ware',
    '51': 'Information',
    '52': 'Finance/Insurance',
    '53': 'RE/Rental/Lease',
    '54': 'Prof/Science/Tech',
    '55': 'Mgmt_comp',
    '56': 'Admin_sup/Waste_Mgmt_Rem',
    '61': 'Educational',
    '62': 'Healthcare/Social_assist',
    '71': 'Arts/Entertain/Rec',
    '72': 'Accom/Food_serv',
    '81': 'Other_no_pub',
    '92': 'Public_Admin'
})
df['NAICS']

0                     Retail_trade
1                  Accom/Food_serv
2         Healthcare/Social_assist
3                    Manufacturing
4                     Other_no_pub
                    ...           
691393               Manufacturing
691394               Manufacturing
691395                Retail_trade
691396                Retail_trade
691397               Manufacturing
Name: NAICS, Length: 691398, dtype: object

#### Now the column NAICS is a category feature. 

#### Since the term of loan is a function of the expected life-time of the assets, loans backed by real estate will have terms 20 years or greater (>=240 months). Therefore we decided to create a column named 'RealEstate' based on the column Term. If Term >=240 months then RealEstate = 1 else 0. 

In [43]:
df['RealEstate'] = df['Term'].apply(lambda x: 1 if x >= 240 else 0)
df.drop('Term',inplace=True,axis=1)


In [44]:
df.head(20)

,State,NAICS,NoEmp,NewExist,LowDoc,MIS_Status,SBA_GrPortion,RealEstate
0,IN,Retail_trade,4,1.0,Y,P I F,0.80,0
1,IN,Accom/Food_serv,2,1.0,Y,P I F,0.80,0
2,IN,Healthcare/Social_assist,7,0.0,N,P I F,0.75,0
3,CT,Manufacturing,19,0.0,N,P I F,0.75,0
4,FL,Other_no_pub,1,1.0,Y,P I F,0.80,0
5,FL,Accom/Food_serv,2,1.0,N,P I F,0.75,1
6,NC,Other_no_pub,1,1.0,Y,P I F,0.80,0
7,IL,Construction,24,0.0,N,P I F,0.75,0
8,NC,Retail_trade,2,1.0,N,P I F,0.75,0
9,OK,Wholesale_trade,5,1.0,N,P I F,0.75,0


### Now the column State, NAICS, LowDoc, MIS_Status is in category form. 


In [55]:
def information(df):
    list_item = []
    for col in df.columns:
          list_item.append([col,df[col].nunique(), list(df[col].sample(5).drop_duplicates().values)])
    dfDesc = pd.DataFrame(columns=['Feature','unique','values'],data=list_item)
    print(dfDesc)
information(df[['State','NAICS','LowDoc','MIS_Status']])

      Feature  unique                                             values
0       State      52                               [CA, LA, MA, TX, CO]
1       NAICS      20  [Healthcare/Social_assist, Accom/Food_serv, Re...
2      LowDoc       2                                             [N, Y]
3  MIS_Status       2                                            [P I F]


#### After doing some calculation, there are 52 unique values in column State and 20 unique values in column NAICS which is a large number. Therefore we decided to use Binary Encode for tranforming these feature into numerical form.

In [46]:
from category_encoders import BinaryEncoder
columns_to_encode=['State','NAICS','LowDoc','MIS_Status']
encoder = BinaryEncoder(cols=columns_to_encode)
df_encoded = encoder.fit_transform(df[columns_to_encode])
df_final = pd.concat([df.drop(columns=columns_to_encode), df_encoded], axis=1)
df_final.to_csv('../Data/data_binary_encoded.csv', index=False)

In [47]:
df_final

,NoEmp,NewExist,SBA_GrPortion,RealEstate,State_0,State_1,State_2,State_3,State_4,State_5,NAICS_0,NAICS_1,NAICS_2,NAICS_3,NAICS_4,LowDoc_0,LowDoc_1,MIS_Status_0,MIS_Status_1
0,4,1.0,0.80,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,1
1,2,1.0,0.80,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1
2,7,0.0,0.75,0,0,0,0,0,0,1,0,0,0,1,1,1,0,0,1
3,19,0.0,0.75,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1
4,1,1.0,0.80,0,0,0,0,0,1,1,0,0,1,0,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
691393,20,0.0,0.50,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1
691394,40,0.0,0.75,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1
691395,6,0.0,0.80,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,1
691396,6,0.0,0.50,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,1
